In [ ]:
from utils import get_clues, generate_abs_path
from src.opensource.llama_frontend import chat_pipeline
import pandas as pd
import os
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from src.opensource.templates import SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE
from const import STATE_CLUES_NOTES_DICT

In [2]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [3]:
inst_template = PromptTemplate.from_template(INST_LLAMA_TEMPLATE)
inst_template = inst_template.format(state='Jharkhand')
print(inst_template)


    [INST] Name the object on the basis of the above clues from Jharkhand. After your guess, I will let you know if you are correct or not. If you're correct, do not make any further guesses and end the conversation. If you are wrong, make your second and final guess. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here.[/INST]



In [4]:
# from utils.utils import state_clue_notes_dict


In [ ]:
df = pd.read_csv('/home/t-sahuja/cultural_artifacts/clues/jharkhand/artifact_clues.csv')

In [ ]:
df.iloc[4]['artifact']

In [ ]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [5]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row["clues"].strip().split("\n")
        artifact = row["artifact"].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue.strip()}\n"
            fin_clues = output.strip()

        agent = chat_pipeline(
            clue_list=fin_clues,
            prompt_text=SYS_LLAMA_TEMPLATE,
            conversation_buffer=conversation_buffer,
        )
        guess1 = agent.predict(human_input=inst_template)
        guess1 = guess1.split(":")[1].strip().lower()
        if artifact in guess1:
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": "NA",
                    "ground_truth": artifact,
                    "clues": row["clues"].strip(),
                },
                ignore_index=True,
            )
            conversation_buffer.clear()
            continue
        else:
            guess2 = agent.predict(human_input="Your first guess is not correct")
            guess2 = guess2.split(":")[1].strip().lower()
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": guess2,
                    "ground_truth": artifact,
                    "clues": row["clues"].strip(),
                },
                ignore_index=True,
            )
            conversation_buffer.clear()
    return df_eval

In [6]:
conversation_buffer.clear()

In [7]:
df_eval = get_outputs(df, df_eval)

NameError: name 'df' is not defined

In [ ]:
gg = generate_abs_path('results/opensource/llama')
os.path.exists(gg)

In [ ]:
df_eval.to_csv('jharkhand_evals_with_state_name.csv', index=False)

In [15]:
def compile_results(STATE_CLUES_NOTES_DICT, output_dir):
    for key, val in STATE_CLUES_NOTES_DICT.items():
        curr_path = os.path.join(output_dir,key)
        if not os.path.exists(curr_path):
            os.makedirs(curr_path)
        
        clue_path = val[0]
        notes_path = val[1] if len(val) > 1 else None
        
#         print("Getting results for {key}ate")
        print(f"getting results for {key} state")
        conversation_buffer.clear()
        df_clues_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])
        df_clues = pd.read_csv(clue_path)
        
        
        print(f"Running clues eval for {key} state")
        clues_result_path = os.path.join(curr_path,'eval_clues.csv')
        if not os.path.exists(clues_result_path):
            df_clues_eval = get_outputs(df_clues, df_clues_eval)
            df_clues_eval.to_csv(clues_result_path, index=False)
        else:
            print(f"Clue eval results already exist for {key} state")
        
        if notes_path:
            conversation_buffer.clear()
            df_notes = pd.read_csv(notes_path)
            df_notes_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])
            notes_result_path = os.path.join(curr_path,'eval_notes.csv')
            if not os.path.exists(notes_result_path):
                print(f"Running notes eval for {key} state")
                df_notes_eval = get_outputs(df_clues, df_clues_eval)
                df_notes_eval.to_csv(notes_result_path, index=False)
            else:
                print(f"Notes eval results already exist for {key} state")
                
            

In [ ]:
compile_results(STATE_CLUES_NOTES_DICT, '/home/t-sahuja/cultural_artifacts/results/opensource/llama')

getting results for jharkhand state
Running clues eval for jharkhand state
Clue eval results already exist for jharkhand state
Running notes eval for jharkhand state
artifact--0---
artifact--1---
artifact--2---
artifact--3---
artifact--4---
artifact--5---
artifact--6---
artifact--7---
artifact--8---
artifact--9---
artifact--10---
artifact--11---
artifact--12---
artifact--13---
artifact--14---
artifact--15---
artifact--16---
artifact--17---
artifact--18---
artifact--19---
artifact--20---
getting results for assam state
Running clues eval for assam state
Clue eval results already exist for assam state
Running notes eval for assam state
artifact--0---
artifact--1---
artifact--2---
artifact--3---
artifact--4---
artifact--5---
artifact--6---
artifact--7---
artifact--8---
artifact--9---
artifact--10---
getting results for gujarat state
Running clues eval for gujarat state
Clue eval results already exist for gujarat state
getting results for karnataka state
Running clues eval for karnataka st